In [3]:
import numpy as np
import json

In [4]:
Pattern = {0: ["1","0","1"], 1: ["0","1","0"], 2: ["1","0","0"]}
N = len(Pattern)
neighbors = np.array([[i for i in range(N)] for j in range(N)])

In [5]:
def filter_pattern(pattern, f):
    f_pat = {}
    for i in pattern.keys():
        if f[i]:
            f_pat[i] = pattern[i]
    return f_pat

def combinations(n, k):
    combos = []
    if (k == 0):
        return [[]]
    elif (k == 1):
        return [[i] for i in n] 
    for i in range(len(n)): 
        head = n[i:i+1]

        tail = combinations(n[i+1:],k-1)

        for j in range(len(tail)):
            #print("tail[j]", tail[j])
            if (type(tail[j]) == int):
                combo = head + [tail[j]]
            else:
                combo = head + tail[j]
            combos.append(combo)   
    return combos

def get_strategy(pattern, idx, known_idx):
    my_neighb = known_idx[idx].tolist() #np.delete(known_idx[idx],np.where(known_idx[idx]==idx)[0]).tolist()
    N = len(pattern)
    l = len(pattern[idx])
    #print(my_neighb)

    for Nneigh in range(1,len(my_neighb)+1):
        neighb_lists = combinations(my_neighb, Nneigh)
        #print("neigh lists", neighb_lists)

        #neighb_lists = [[idx] + nl for nl in neighb_lists]
        
        for n in neighb_lists:
            columns = sorted(n)
            mask = [(i in columns) for i in range(N)]
            #print("mask", mask)
            f_pat = filter_pattern(pattern, mask) #Leave just the observed data columns
            #print("f_pat", f_pat) 
            
            #print(idx,pattern,f_pat)
            strat = {}
            test = True
            for i in range(l):
                p = ''.join(f_pat[a][i] for a in columns)
                #print(p)
                if p not in strat:
                    strat[p] = pattern[idx][(i+1)%l]
                else:
                    if strat[p] != pattern[idx][(i+1)%l]:
                        test = False
                        break
        
            if test: 
                same = True
                ini = strat[list(strat.keys())[0]]
                for p in strat:
                    same = same and (strat[p] == ini)
                if same: #If it takes always the same action...
                    return (), {'any': strat[list(strat.keys())[0]]}
                else:
                    return tuple(columns), strat
    return None  #If the output is None, then most likely there is no stable pattern

In [6]:
rel_data = {}
pat = Pattern
for i in range(N): 
    rel_data[i] = {}
    rel_data[i]["pattern"] = pat[i]
    strat = get_strategy(pat, i, neighbors)
    rel_data[i]["neigh"] = strat[0]
    rel_data[i]["strat"] = strat[1]

In [8]:
with open(f'graph_data_N{N:d}.json', 'w') as json_file:
  json.dump(rel_data, json_file)